In [19]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg

from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries

In [20]:
data = pd.read_csv('20231031.csv',encoding='utf-8')
data = data[~data['仓库'].isin(['8013','8021','8025','8031'])]
# data =data[data['零件号']!='6RD959801E']
data['日期'] = pd.to_datetime(data['日期'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
new_df = data.groupby(['零件号','日期','仓库'])['需求数量'].sum().reset_index()
new_df['日期'] = pd.to_datetime(new_df['日期'])

In [21]:
new_df.loc[(new_df['零件号'] == '6RD959801E') & (new_df['日期'] == '2023/3/27')& 
           (new_df['仓库'] == '1000-1'), '需求数量'] = 6

In [22]:
data_des = new_df.copy()
data_des["year"] = pd.to_datetime(data_des['日期']).dt.year.astype(int)
data_des["month"] = pd.to_datetime(data_des['日期']).dt.month.astype(int)
real_sum_counts = data_des.groupby(['零件号','仓库','year','month']).sum().reset_index()
real_sum_counts.sort_values(by=['零件号','仓库','year','month']).head(2)

,零件号,仓库,year,month,需求数量
0,11D941078C,1000-2,2023,1,1
1,11D941078C,1000-2,2023,2,3


In [23]:
# real_sum_counts[(real_sum_counts['零件号']=='3CC945208A')&(real_sum_counts['仓库']=='1000-1')].tail(10)

In [24]:
# 生成日期范围
date_range = pd.date_range(start='2021-01-01', end='2023-10-31', freq='D')
# 创建空的DataFrame，准备存储填充后的结果
filled_df = pd.DataFrame()

# 针对每个 SKU 进行填充操作
for sku, group in new_df.groupby(['零件号','仓库']):
    sku_group = group.set_index('日期').reindex(date_range, fill_value=0).reset_index()
    sku_group['零件号'] = sku[0]
    sku_group['仓库'] = sku[1]
    filled_df = filled_df.append(sku_group, ignore_index=True)

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_50595/3238187233.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filled_df = filled_df.append(sku_group, ignore_index=True)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_50595/3238187233.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filled_df = filled_df.append(sku_group, ignore_index=True)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_50595/3238187233.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filled_df = filled_df.append(sku_group, ignore_index=True)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_50595/3238187233.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

In [25]:
import calendar  
import datetime  
  

    
def feature_processing(data,end_time):
    data['合并列'] = data['零件号'] + '_' + data['仓库']
    # 去除多余的列名
    data = data.drop(columns=['零件号', '仓库'])
    data = data.set_index(['index', '合并列'])['需求数量'].unstack()
    data.columns.name = None
    data = data.reset_index()
    new_df = data.copy().rename(columns={'index': 'date'})
    new_df['date'] = pd.to_datetime(new_df['date'], format='%Y-%m-%d')
    new_df = new_df.set_index('date')
    new_df = new_df.asfreq('D')
    new_df = new_df.sort_index()
    data_train = new_df[new_df.index <= end_time].copy()
    return data_train


def demods_groby_month(data):
    """天级别聚合month"""
    data_prs = data.reset_index()
    data_prs = data_prs.rename(columns={'index': 'date'})
    data_prs["year"] = pd.to_datetime(data_prs['date']).dt.year.astype(int)
    data_prs["month"] = pd.to_datetime(data_prs['date']).dt.month.astype(int)
    data_prs_info = data_prs.groupby(['year', 'month']).sum().reset_index()
    data_prs_info = data_prs_info.set_index(['year', 'month']).stack()
    data_prs_info = data_prs_info.rename_axis(index=['year', 'month', '零件号'])
    data_prs_info = data_prs_info.reset_index()
    data_prs_info[['零件号', '仓库代码']] = data_prs_info['零件号'].str.split('_', expand=True).reset_index(drop=True)
    data_prs_info.columns =['year','month','零件号','pred_values','仓库']
    return data_prs_info


def find_outliers_3sigma(data):  
    
    mean = sum(data) / len(data)  
    std_dev = (sum((x - mean) ** 2 for x in data) / len(data)) ** 0.5  
    if mean<=150:
        thr_sig_num_high  = mean + 3 * std_dev
    else:
        thr_sig_num_high  = mean
        
    thr_sig_num_low  = mean - 1 * std_dev
#     q3 = lambda x: x.quantile(0.75)  
#     q1 = lambda x: x.quantile(0.25)  
#     thr_sig_num_high = q3(x) + 1.5 * (q3(x) - q1(x)) 
#     thr_sig_num_low = q1(x) - 1.5 * (q3(x) - q1(x)) 

    return thr_sig_num_high,thr_sig_num_low



# 生成一个日期对象，表示2023年1月1日  
start_date = datetime.datetime(2023, 1, 1)  
  
# 初始化一个空列表来保存每个月的最后一天  
month_end_dates = []  
  
# 使用 calendar.monthrange() 函数获取每个月的天数  
# 然后加1天，因为我们想要的是当月的最后一天  
for month in range(2, 8):  # 这里 13 是因为一年只有12个月  
    _, num_days = calendar.monthrange(2023, month)  # 获取2023年的日历  
    end_date = start_date.replace(month=month, day=num_days)  # 创建日期对象  
    month_end_dates.append(end_date)  # 添加到列表中  
pred_full_pp = pd.DataFrame()

for date in month_end_dates:  
    end_train = date.strftime("%Y-%m-%d")
    data_train = feature_processing(filled_df,date)
    sku_list = data_train.columns
    # 创建空的字典，用于存储每个时间序列的预测模型
    forecasters = {}
    # 循环遍历每批次SKU，分别训练预测模型
    for i in range(0, len(sku_list), 200):
        batch_skus = sku_list[i:i + 200]
        for sku in batch_skus:
            forecaster = ForecasterAutoreg(  
            regressor=Ridge(random_state=123),  
            lags=30,  
                    )
            # 拟合模型
            forecaster.fit(y=data_train[sku])
            forecasters[sku] = forecaster
        # print(f"Finished training batch {i // self.batch_size + 1}/{len(sku_list) // self.batch_size + 1}")
    # 进行未来预测
    predictions = pd.DataFrame()
    for sku, forecaster in forecasters.items():
        forecast = forecaster.predict(steps=30)
        predictions[sku] = forecast
    predictions[predictions < 0.1] = 0


    his_info = demods_groby_month(data_train)
    thr_std = his_info.groupby(['零件号','仓库'])['pred_values'].apply(find_outliers_3sigma).reset_index()
    thr_std['pred_values_high']= [x[0] for x in thr_std['pred_values']]
    thr_std['pred_values_lower']= [x[1] for x in thr_std['pred_values']]
    for i in ['pred_values_high','pred_values_lower']:
        thr_std[i] =[0 if x<0 else x for x in thr_std[i]]
    thr_std = thr_std.drop(['pred_values'],axis =1 )
    
    

    pred_info = demods_groby_month(predictions)
    pred_info['pred_values'] = pred_info['pred_values'].round(2)
    
    
    full_pred_info = pd.merge(pred_info,thr_std,on =['零件号','仓库'],how ='left')
    full_pred_info['pred_values'] = full_pred_info.apply(lambda row: row['pred_values_high'] 
                                if row['pred_values'] > row['pred_values_high'] 
                                 else (row['pred_values_lower'] if row['pred_values'] < row['pred_values_lower'] 
                                       else row['pred_values']), axis=1)  
#     full_pred_info = full_pred_info.drop(['pred_values_lower','pred_values_high'],axis =1 )
    

    pred_full_pp = pred_full_pp.append(full_pred_info)


pred_full_pp.head()



/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_50595/3861897121.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_full_pp = pred_full_pp.append(full_pred_info)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_50595/3861897121.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_full_pp = pred_full_pp.append(full_pred_info)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_50595/3861897121.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_full_pp = pred_full_pp.append(full_pred_info)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_50595/3861897121.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. 

,year,month,零件号,pred_values,仓库,pred_values_high,pred_values_lower
0,2023,3,11D941078C,1.956211,1000-2,1.956211,0.0
1,2023,3,3CC945208A,1.170000,1000-1,2.233670,0.0
2,2023,3,3CC945208A,0.000000,3001,0.615385,0.0
3,2023,3,3CC945208A,0.000000,5001,0.615385,0.0
4,2023,3,3CC945208A,0.000000,5002,0.000000,0.0


In [26]:
compar_pred_real_info = pd.merge(pred_full_pp,real_sum_counts,on =['零件号','仓库','year','month'],how ='left')
compar_pred_real_info.fillna(0,inplace=True)

In [27]:
svg_pred =pd.read_csv('svg预测结果.csv',encoding='gb18030', header=1)
svg_pred['日期'] = pd.to_datetime(svg_pred['日期'], format='%Y%m').dt.strftime('%Y-%m')
svg_pred["year"] = pd.to_datetime(svg_pred['日期']).dt.year.astype(int)
svg_pred["month"] = pd.to_datetime(svg_pred['日期']).dt.month.astype(int)
svg_pred = svg_pred.groupby(['零件代码','仓库','year','month'])['预测值'].sum().reset_index()
svg_pred = svg_pred.rename(columns ={"零件代码":"零件号"})

In [28]:
full_compart_info  = pd.merge(compar_pred_real_info,svg_pred,on = ['零件号','仓库','year','month'],how ='left')

In [29]:
full_compart_info.fillna(0,inplace =True)

full_compart_info['需求数量']= [1.01 if x ==0 else x for x in full_compart_info['需求数量']]

full_compart_info['pred_values']= [1 if x ==0 else x for x in full_compart_info['pred_values']]

full_compart_info['chumi_mape'] = (abs(full_compart_info['需求数量'] - full_compart_info['pred_values']) 
                                 / full_compart_info['需求数量']) 
full_compart_info['svg_mape'] = (abs(full_compart_info['需求数量'] - full_compart_info['预测值']) 
                                 / full_compart_info['需求数量']) 
full_compart_info.sort_values(['零件号']).tail(50)

,year,month,零件号,pred_values,仓库,pred_values_high,pred_values_lower,需求数量,预测值,chumi_mape,svg_mape
54,2023,7,3CC945208A,1.000000,8011,1.287434,0.000000,2.00,0.833333,0.500000,0.583333
55,2023,7,3CC945208A,1.000000,8012,1.143700,0.000000,1.00,2.000000,0.000000,1.000000
56,2023,7,3CC945208A,0.140000,8015,1.526814,0.000000,4.00,1.066667,0.965000,0.733333
57,2023,7,3CC945208A,1.000000,8022,1.000000,0.000000,4.00,0.428571,0.750000,0.892857
58,2023,7,3CC945208A,0.760000,8023,5.808764,0.000000,13.00,3.464286,0.941538,0.733516
66,2023,8,3CC945208A,0.530000,8011,1.703617,0.000000,4.00,1.300000,0.867500,0.675000
61,2023,8,3CC945208A,7.370000,1000-1,10.729112,0.000000,8.00,7.000000,0.078750,0.125000
62,2023,8,3CC945208A,5.417966,3001,5.417966,0.000000,5.00,3.904762,0.083593,0.219048
63,2023,8,3CC945208A,2.840000,5001,4.779976,0.000000,9.00,3.142857,0.684444,0.650794
70,2023,8,3CC945208A,9.326898,8023,9.326898,0.000000,14.00,5.583333,0.333793,0.601191


In [30]:
full_compart_info.sort_values(['零件号','month']).to_excel('预测效果_20231101.xlsx')

In [31]:
full_compart_info.chumi_mape.mean()

1.2291404958326362

In [32]:
full_compart_info.svg_mape.mean()


# 1.提高模型响应速度 最近月份的比重
# 2.对话机器人的流程 chatgpt 

1.5952113465501807

In [33]:
full_compart_info.groupby(['month'])['chumi_mape','svg_mape'].mean().reset_index()

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_50595/433864545.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  full_compart_info.groupby(['month'])['chumi_mape','svg_mape'].mean().reset_index()


,month,chumi_mape,svg_mape
0,3,0.159376,0.798175
1,4,0.698064,1.800608
2,5,0.467168,1.039344
3,6,0.533929,0.952562
4,7,3.965394,3.789389
5,8,1.550911,1.191190


In [34]:

def feature_processing(data,end_time):
    data['合并列'] = data['零件号'] + '_' + data['仓库']
    # 去除多余的列名
    data = data.drop(columns=['零件号', '仓库'])
    data = data.set_index(['index', '合并列'])['需求数量'].unstack()
    data.columns.name = None
    data = data.reset_index()
    new_df = data.copy().rename(columns={'index': 'date'})
    new_df['date'] = pd.to_datetime(new_df['date'], format='%Y-%m-%d')
    new_df = new_df.set_index('date')
    new_df = new_df.asfreq('D')
    new_df = new_df.sort_index()
    data_train = new_df[new_df.index <= end_time].copy()
    return data_train


def demods_groby_month(data):
    """天级别聚合month"""
    data_prs = data.reset_index()
    data_prs = data_prs.rename(columns={'index': 'date'})
    data_prs["year"] = pd.to_datetime(data_prs['date']).dt.year.astype(int)
    data_prs["month"] = pd.to_datetime(data_prs['date']).dt.month.astype(int)
    data_prs_info = data_prs.groupby(['year', 'month']).sum().reset_index()
    data_prs_info = data_prs_info.set_index(['year', 'month']).stack()
    data_prs_info = data_prs_info.rename_axis(index=['year', 'month', '零件号'])
    data_prs_info = data_prs_info.reset_index()
    data_prs_info[['零件号', '仓库代码']] = data_prs_info['零件号'].str.split('_', expand=True).reset_index(drop=True)
    data_prs_info.columns =['year','month','零件号','pred_values','仓库']
    return data_prs_info


def find_outliers_3sigma(data):  

    mean = sum(data) / len(data)  
    std_dev = (sum((x - mean) ** 2 for x in data) / len(data)) ** 0.5  
    thr_sig_num_high  = mean + 3 * std_dev
    thr_sig_num_low  = mean - 1 * std_dev

#     q3 = lambda x: x.quantile(0.75)  
#     q1 = lambda x: x.quantile(0.25)  
#     thr_sig_num_high = q3(x) + 1.5 * (q3(x) - q1(x)) 
#     thr_sig_num_low = q1(x) - 1.5 * (q3(x) - q1(x)) 

    return thr_sig_num_high,thr_sig_num_low

test  = feature_processing(filled_df,'2023-03-31')

test = demods_groby_month(test)


output = his_info.groupby(['零件号','仓库'])['pred_values'].apply(find_outliers_3sigma).reset_index()
output['pred_values_high']= [x[0] for x in output['pred_values']]
output['pred_values_lower']= [x[1] for x in output['pred_values']]
output


,零件号,仓库,pred_values,pred_values_high,pred_values_lower
0,11D941078C,1000-2,"(4.346080684583596, -0.7175107658289408)",4.346081,-0.717511
1,3CC945208A,1000-1,"(10.729111907269315, -1.8129297755413847)",10.729112,-1.812930
2,3CC945208A,3001,"(5.417966048545466, -1.0748058871495643)",5.417966,-1.074806
3,3CC945208A,5001,"(4.779976343692022, -0.8621426521984157)",4.779976,-0.862143
4,3CC945208A,5002,"(0.5623121524243544, -0.1444266314532794)",0.562312,-0.144427
5,3CC945208A,6000,"(3.3595176831471014, -0.6467209481458079)",3.359518,-0.646721
6,3CC945208A,8011,"(1.7036171726299252, -0.3528186274357815)",1.703617,-0.352819
7,3CC945208A,8012,"(1.2661012232510875, -0.2930014830191797)",1.266101,-0.293001
8,3CC945208A,8015,"(2.7231880819309424, -0.5206325864500989)",2.723188,-0.520633
9,3CC945208A,8022,"(2.474635491346242, -0.5238032282982098)",2.474635,-0.523803


In [35]:
test[(test['零件号']=='3CC945208A')&(test['仓库']=='5001')]

,year,month,零件号,pred_values,仓库
3,2021,1,3CC945208A,0,5001
15,2021,2,3CC945208A,0,5001
27,2021,3,3CC945208A,0,5001
39,2021,4,3CC945208A,0,5001
51,2021,5,3CC945208A,0,5001
63,2021,6,3CC945208A,0,5001
75,2021,7,3CC945208A,0,5001
87,2021,8,3CC945208A,0,5001
99,2021,9,3CC945208A,0,5001
111,2021,10,3CC945208A,0,5001


In [36]:
real_sum_counts[(real_sum_counts['零件号']=='3CC945208A')&(real_sum_counts['仓库']=='5001')]

,零件号,仓库,year,month,需求数量
28,3CC945208A,5001,2023,2,1
29,3CC945208A,5001,2023,3,1
30,3CC945208A,5001,2023,4,6
31,3CC945208A,5001,2023,5,4
32,3CC945208A,5001,2023,6,1
33,3CC945208A,5001,2023,7,4
34,3CC945208A,5001,2023,8,9
35,3CC945208A,5001,2023,9,18
36,3CC945208A,5001,2023,10,13
